In [68]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import sys
sys.path.append('../../acd/acd')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import os
vector_cache = os.path.join(os.getcwd(), '.vector_cache/input_vectors.pt')
word_vectors ='glove.6B.300d'
batch_size=  42


In [70]:
def makedirs(name):
    """helper function for python 2 and 3 to call os.makedirs()
       avoiding an error if the directory to be created already exists"""

    import os, errno

    try:
        os.makedirs(name)
    except OSError as ex:
        if ex.errno == errno.EEXIST and os.path.isdir(name):
            # ignore existing directory
            pass
        else:
            # a different error happened
            raise



In [71]:
sst_pkl = pickle.load(open('../../acd/dsets/sst/sst.pkl', 'rb'))

In [72]:
from os.path import isdir
sys.path.append('../../acd/visualization')
sys.path.append('../../acd/acd/util')
sys.path.append('../../acd/acd/scores')
sys.path.append('../../acd/acd/agglomeration')
import viz_1d as viz
import tiling_1d as tiling
import agg_1d as agg
import cd
import score_funcs

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [73]:
batch_size

42

In [74]:
# form class to hold data
class B:
    text = torch.zeros(1).to(device)
sys.path.append('../../acd')
sys.path.append('../../acd/dsets/sst')
from dsets.sst import dset
from dsets.sst.model import LSTMSentiment

# check out how two models differ
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets

In [75]:

inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)

train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label != 'neutral')

inputs.build_vocab(train, dev, test)
if word_vectors:
    if os.path.isfile(vector_cache):
        inputs.vocab.vectors = torch.load(vector_cache)
    else:
        inputs.vocab.load_vectors(word_vectors)
        makedirs(os.path.dirname(vector_cache))
        torch.save(inputs.vocab.vectors,vector_cache)
answers.build_vocab(train)

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, device=torch.device(0))

In [76]:

model1 = torch.load('../models/model2.pt', map_location=torch.device(0))
model2 = torch.load('../models/model1.pt', map_location=torch.device(0))

/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'model.LSTMSentiment' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [77]:
model1.eval()
model2.eval()

LSTMSentiment(
  (embed): Embedding(18844, 300)
  (lstm): LSTM(300, 128)
  (hidden_to_label): Linear(in_features=128, out_features=2, bias=True)
)

In [78]:
dev_batch.text.shape

torch.Size([15, 42])

In [79]:
torch.zeros(4,3)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [80]:
torch.transpose(model1.embed(dev_batch.text), 1,2).shape

torch.Size([15, 300, 42])

In [39]:
model1.hidden_to_label.bias.data[:,None]

tensor([[ 0.0671],
        [-0.0113]], device='cuda:0')

In [48]:
test_out_rel, test_out_irrel = cd.cd_batch_text(dev_batch, model1, 2,4)

In [50]:
test_comparison = model1.forward(dev_batch)- model1.hidden_to_label.bias.data

In [60]:
test_out_rel.shape

torch.Size([2, 42])

In [67]:
cd.cd_penalty(dev_batch, model1, model2, 0,5)

tensor(0.1072, device='cuda:0')

In [54]:
test_out_irrel.=

torch.Size([2, 42])

In [58]:
torch.abs((torch.transpose(test_comparison,0,1) - (test_out_rel +test_out_irrel))).mean()

tensor(1.6533e-07, device='cuda:0', grad_fn=<MeanBackward1>)

In [81]:

# check out how two models differ
import torch.optim as O
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
n_dev_correct, dev_loss = 0, 0
cur_idx = 0
for dev_batch_idx, dev_batch in enumerate(dev_iter):
    print(dev_batch.text.shape)
    if dev_batch_idx >5:
        break
    print(cd.cd_penalty(dev_batch, model1, model2, 0, 2))
    

torch.Size([6, 42])
tensor(0.0336, device='cuda:0')
torch.Size([8, 42])
tensor(0.0890, device='cuda:0')
torch.Size([10, 42])
tensor(0.0730, device='cuda:0')
torch.Size([11, 42])
tensor(0.0856, device='cuda:0')
torch.Size([12, 42])
tensor(0.0798, device='cuda:0')
torch.Size([14, 42])
tensor(0.0895, device='cuda:0')
torch.Size([15, 42])


In [87]:
a = cd.cd_batch_text(dev_batch, model1, 0,5)

In [96]:
torch.cat((a[0], a[1]),-1).shape

torch.Size([2, 84])

In [113]:
cd.cd_penalty(dev_batch, model1, model2, 0,5)

tensor(6.0455, device='cuda:0')

In [ ]:
torch.nn.functional.softmax(a, dim =0)

In [25]:
start = np.random.randint(batch.text.shape[0])

In [26]:
start

26

In [28]:
start = np.random.randint(batch.text.shape[0]-1)

In [40]:
batch_length = batch.text.shape[0]
if batch_length >2:
    start = np.random.randint(batch_length-1)
    stop = start + np.random.randint(batch_length-start)

In [39]:
np.random.randint((batch.text.shape[0])-start)

10

In [41]:
start

32

In [43]:
stop

41

In [35]:
np.random.randint((batch.text.shape[0])-start)

0

In [32]:
dev_batch.text.shape

torch.Size([47, 1])

In [33]:
start


44

In [34]:
stop

44

In [22]:
batch =dev_batch

In [17]:
dev_batch.text.shape[0]

47

In [ ]:
with torch.no_grad():
# check out how two models differ
    import torch.optim as O
    import torch.nn as nn
    criterion = nn.CrossEntropyLoss()
    n_dev_correct, dev_loss = 0, 0
    cur_idx = 0
    for dev_batch_idx, dev_batch in enumerate(dev_iter):
        if dev_batch_idx > 200:
            break

        cd.cd_text(dev_batch,model1, 0,dev_batch.text.shape[0])



In [ ]:
model2.forward(dev_batch) - model2.hidden_to_label.bias.data

In [ ]:
rel ,irrel = cd.cd_text(dev_batch, model2, 0,5)

In [ ]:
rel

In [ ]:
[sst_pkl["itos"][x] for x in dev_batch.text.data]

In [ ]:
model1.forward(dev_batch) - model1.hidden_to_label.bias.data

In [1]:
model1

NameError: name 'model1' is not defined

In [ ]:
rel ,irrel = cd.cd_text(dev_batch, model1, 0,3)

In [ ]:
rel +irrel

In [ ]:
# fuck, we do it livea